In [15]:
# Add module import path to system path.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Hot-loading modules before executing code: allows us to write code while testing it in this notebook.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import logging

# Logging strategy for this notebook.
root = logging.getLogger()
root.setLevel(logging.INFO)

if not root.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)

    root.addHandler(handler)

In [69]:
from parser import Parser

parser = Parser()

In [70]:
teams = parser.parse_teams()

2022-03-06 21:29:45,029 - parser - INFO - Column names are TeamID, TeamName, FirstD1Season, LastD1Season
2022-03-06 21:29:45,031 - parser - INFO - Processed 372 teams.


In [71]:
parser.parse()

2022-03-06 21:29:45,329 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-06 21:29:45,870 - parser - INFO - Processed 174471 games.
2022-03-06 21:29:45,871 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-06 21:29:45,878 - parser - INFO - Processed 2317 games.
2022-03-06 21:29:45,879 - parser - INFO - Column names are Season, DayZero, RegionW, RegionX, RegionY, RegionZ
2022-03-06 21:29:45,880 - parser - INFO - Processed 38 seasons.
2022-03-06 21:29:45,934 - parser - INFO - [[0.6363636363636364, 0.6666666666666666], [0.6666666666666666, 0.6363636363636364], [0.6206896551724138, 0.68], [0.68, 0.6206896551724138], [0.9259259259259259, 0.3793103448275862], [0.3793103448275862, 0.9259259259259259], [0.7407407407407407, 0.6785714285714286], [0.6785714285714286, 0.7407407407407407], [0.7666666666666667, 0.7407407407407407], [0.7407407407407407, 0.7666666666666667], [0.571428571428571